In [11]:
import numpy as np
import pandas as pd
import scipy as sp
import scanpy as sc
import squidpy as sq
import anndata as ad
import seaborn as sns
import simstpy as sim
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 
import statsmodels.api as sm
import time
import os
import gpflow
os.chdir(os.path.expanduser('~/GPcounts/GPcounts'))
from GPcounts.RNA_seq_GP import rna_seq_gp


In [12]:
adata = sc.read_h5ad(r"C:\Users\User\GPcounts\GPcounts\demo_notebooks\2000 spots inducing benchmark")

Y = pd.DataFrame(data=adata.X.todense().astype(int).copy(), 
                      index=adata.obs_names, 
                      columns=adata.var_names)

spatial_locations = pd.DataFrame(data=adata.obsm['spatial'], 
                                     index=adata.obs_names, 
                                     columns=['x', 'y'])

spatial_locations['total_counts'] = adata.X.sum(axis=1)

scales = []
for i in range(0, len(Y.columns)):
    model=smf.glm(formula = "Y.iloc[:, i]~0+spatial_locations['total_counts']", 
                    data=Y, 
                    family=sm.families.NegativeBinomial(sm.families.links.identity())).fit()
    res = model.params[0]*spatial_locations['total_counts']
    scales.append(res)

scalesdf=pd.DataFrame(scales)
scalesdf=scalesdf.T

Y = Y.T
X = spatial_locations[['x', 'y']]

c:\Users\User\GPcounts\.venv\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The identity link alias is deprecated. Use Identity instead. The identity link alias will be removed after the 0.15.0 release.
  warnings.warn(
c:\Users\User\GPcounts\.venv\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
c:\Users\User\GPcounts\.venv\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:308: DomainWarning: The identity link function does not respect the domain of the NegativeBinomial family.
  warnings.warn((f"The {type(family.link).__name__} link function "
c:\Users\User\GPcounts\.venv\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, cate

In [15]:
# 0.5% len inducing points

M = int(
    (0.5 * (len(X))) / 100
)

gp_counts = rna_seq_gp(X, Y, sparse=True, M=M, scale=scalesdf, safe_mode=False)


log_likelihood_ratio = gp_counts.One_sample_test("Negative_binomial")



  0%|                                                                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

closure = <tensorflow.python.eager.polymorphic_function.polymorphic_function.Function object at 0x000001FA7FDC0AF0>
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 30911.751870400192
        x: [ 1.113e+01  7.247e+00 ... -4.242e+01  4.031e+02]
      nit: 17
      jac: [ 1.113e+01  7.247e+00 ...  9.481e-19  4.895e+00]
     nfev: 52
     njev: 52
 hess_inv: <68x68 LbfgsInvHessProduct with dtype=float64>
tf.Tensor(-30911.751870400192, shape=(), dtype=float64)
lik_alpha = None
alpha: <Parameter: name=softplus, dtype=float64, shape=[], fn="softplus", numpy=403.0521820871244>
lik_alpha = 403.0521820871244
model 1 log likelihood = -30911.751870400192, alpha = 403.0521820871244, km = None
closure = <tensorflow.python.eager.polymorphic_function.polymorphic_function.Function object at 0x000001FA03871DC0>
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 7.031264271417598e+16
        x: [ 3.028e+07

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.95s/it]

lik_alpha = None
alpha: <Parameter: name=softplus, dtype=float64, shape=[], fn="softplus", numpy=16642357108.941544>
lik_alpha = 16642357108.941544
model 1 log likelihood = -7.031264271417598e+16, alpha = 16642357108.941544, km = None
